# Establishing a database connection

### The Engine
The start of any SQLAlchemy application is an object called the Engine. The engine is typically a global object created just once for a particular database server, and is configured using a URL string which will describe how it should connect to the database host or backend.

The Engine is created using the create_engine() method.

### The URL string
The main argument to create_engine() is a URL string. This string indicates to the Engine three important facts:
1. What kind of database are we communicating with? In our example we're using 'mssql' for Microsoft SQL Server.

2. What DBAPI are we using? The Python DBAPI is a third party driver that SQLAlchemy uses to interact with a particular database. In this case, we’re using the name pyodbc, which is an open source Python module for accessing ODBC databases. (ODBC is a standard API for accessing databases.)

3. How do we locate the database? In our case we need to specify the server and database name.

### Getting a connection
The sole purpose of the Engine object from a user-facing perspective is to provide a unit of connectivity to the database called the Connection. When working with the Core directly, the Connection object is how all interaction with the database is done. As the Connection represents an open resource against the database, we want to always limit the scope of our use of this object to a specific context, and the best way to do that is by using Python context manager form, also known as the with statement.

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "localhost"
database_name = "everyloop"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};TrustServerCertificate=yes;UID=sa;PWD=123456;"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=everyloop;TrustServerCertificate=yes;UID=sa;PWD=Kakazayn7; 

Successfully connected to everyloop!


# Query database using raw SQL
 The text() construct allows us to write SQL statements as textual SQL. Rest assured that textual SQL in day-to-day SQLAlchemy use is by far the exception rather than the rule for most tasks, even though it always remains fully available.

 Let's create a simple query to be used in the examples below:

In [2]:
from sqlalchemy import text

query = text("SELECT TOP 5 Id, FirstName, LastName, Email FROM USERS ORDER BY FirstName, LastName")
type(query)

sqlalchemy.sql.elements.TextClause

### The Result Object
When passing our query to Connection.execute() SQLAlchemy will submit the query to the server, which will respond by sending back the requested data (The result set). This result set will be represented as a Result Object returned by Connection.execute() once it gets the response from the server.

Note that it’s best practice to ensure that this object is consumed within the “connect” block, and is not passed along outside of the scope of our connection.

Let's look at a few examples of how to consume the returned result set:

### Execute query and fetch all rows from result

In [3]:
with engine.connect() as conn:
    result = conn.execute(query)

    print(result)
    print(type(result))
    print(result.fetchall())

<class 'sqlalchemy.engine.cursor.CursorResult'>
[('741109-2058', 'Alexander', 'Dahl', 'alexander.dahl@telia.se'), ('530720-7675', 'Alvin', 'Lindholm', 'alvin.lindholm@gmail.com'), ('820624-3075', 'Anders', 'Hansson', 'anders.hansson@hotmail.com'), ('751123-9724', 'Anne', 'Åkerman', 'anne.akerman@hotmail.com'), ('620925-4245', 'Annette', 'Bergfalk', 'annette.bergfalk@telia.se')]


### Execute query and iterate through result

In [4]:
with engine.connect() as conn:
    result = conn.execute(query)

    for row in result:
        print(f"{row.FirstName} {row.LastName}")

Alexander Dahl
Alvin Lindholm
Anders Hansson
Anne Åkerman
Annette Bergfalk


### Iterate column names and fields

In [5]:
with engine.connect() as conn:
    result = conn.execute(query)

    for column_name in result.keys():
        print(column_name.upper().ljust(20), end='')
    
    print()

    for row in result:
        for field in row:
            print(field.ljust(20), end='')
        
        print()

ID                  FIRSTNAME           LASTNAME            EMAIL               
741109-2058         Alexander           Dahl                alexander.dahl@telia.se
530720-7675         Alvin               Lindholm            alvin.lindholm@gmail.com
820624-3075         Anders              Hansson             anders.hansson@hotmail.com
751123-9724         Anne                Åkerman             anne.akerman@hotmail.com
620925-4245         Annette             Bergfalk            annette.bergfalk@telia.se


### Create DataFrames from query result
Pandas and SQLAlchemy are a match made in Python heaven. They're individually amongst Python's most frequently used libraries. Together they're greater than the sum of their parts, thanks to Pandas' built-in SQLAlchemy integration.

In [6]:
import pandas as pd

df = pd.read_sql_query(query, con=engine, index_col="Id")

df

,FirstName,LastName,Email
Id,,,
741109-2058,Alexander,Dahl,alexander.dahl@telia.se
530720-7675,Alvin,Lindholm,alvin.lindholm@gmail.com
820624-3075,Anders,Hansson,anders.hansson@hotmail.com
751123-9724,Anne,Åkerman,anne.akerman@hotmail.com
620925-4245,Annette,Bergfalk,annette.bergfalk@telia.se


# Example search application (unsafe implementation)

In [7]:
search_string = input("Search airports: ")

print(f"Search string: {search_string}\n")

unsafe_query = f"""
select top 10
    IATA, 
    [Airport name] as 'name', 
    [Location served] as 'location' 
from 
    airports 
where 
    [Location served] like '%{search_string}%'"""

with engine.connect() as conn:    
    result = conn.execute(text(unsafe_query))

    print(f"{'IATA'.ljust(8)}{'Airport name'.ljust(50)}{'Location'}")
    ' a'
    for airport in result:
        print(f"{str(airport.IATA).ljust(8)}{str(airport.name).ljust(50)}{airport.location}")

    print(unsafe_query, '\n')


Search string: stock

IATA    Airport name                                      Location
ARN     Stockholm Arlanda Airport                         Stockholm, Sweden
BMA     Stockholm Bromma Airport                          Stockholm, Sweden
FST     Fort Stockton–Pecos County Airport                Fort Stockton, Texas, United States
NYO     Stockholm Skavsta Airport                         Stockholm / Nyköping, Sweden
RLG     Rostock–Laage Airport                             Rostock, Mecklenburg-Vorpommern, Germany
SCK     Stockton Metropolitan Airport                     Stockton, California, United States
SMP     Stockholm Airport[1]                              Stockholm, Papua New Guinea
STO     metropolitan area6                                Stockholm, Sweden
VST     Stockholm Västerås Airport                        Stockholm / Västerås, Sweden

select top 10
    IATA, 
    [Airport name] as 'name', 
    [Location served] as 'location' 
from 
    airports 
where 
    [Location s

### SQL injection
SQL injection is a code injection technique used to attack data-driven applications, in which malicious SQL statements are inserted into an entry field for execution (e.g. to dump the database contents to the attacker). SQL injection is mostly known as an attack vector for websites but can be used to attack any type of SQL database. Even though this vulnerability is known for over 20 years, injections still rank number 3 in the OWASP's Top 10 for web vulnerabilities.

Every programmer involved in the development of applications that makes use of relational databases **SHOULD** know about SQL injection, and how to prevent it. Bad implementations of database interfaces, like the one in the example above, leaves **ALL** your data open for an attack. **ANY** database operation that can be made in a tool like Mangagement Studio can also be made through a web form, or similar user input, using SQL injection.

---

### Concatenation and dynamic queries
The problem with the code above is that it creates a dynamic query by concatenating user inputed data together with predefined SQL. This gives the end-user, not the developer, the final say of what query we send to our DBMS. Consider the example below:

The developer wants the user to be able to provide their name using a web form in order to do a database lookup for information about the user. The developer may think that concatenating the username into the query string before sending the request to the DBMS will solve the problem.

`query = "select * from users where username = '" + username_input + "';"`

This do solve the problem. If the user enters a valid name, the query string may look like this:

`select * from users where username = 'Fredrik';`

But what if the user enters the string '; drop database; -- instead? The query string will then look like this:

`select * from users where username = ''; drop database; --'`

As the application goes on and submits the query it will issue TWO statements to the DBMS: The first will return any row from Users where username is an empty string. The second will delete the database. The two dashes (--) at the end is a line comment to make sure any text after the seccond statement (in this case the ' at the end) is ignored and doesn't cause the query to be invalid.

**YOU MUST NEVER ALLOW USER INPUT IN ANY PART OF YOUR QUERIES!**

---

### Parameterized queries
The DBMS is built to be able to recieve the SQL statement and the user data seperatly. This is done by writting SQL using parameters. 

Here is an example of what that would look like in T-SQL:

`Select * from users where username = @username_input;`

Now we have a static SQL statment that can be sent to the server coupled with the value of the @username_input parameter. This way, the user can't modify the query itself, and we can be assured that the server interprets the query as we intended.

**USER DATA IN QUERIES SHOULD ALWAYS BE PARAMETERIZED!**

---

### Bound parameters in SQLAlchemy

SQLAlchemy's text() construct allows us to bind parameters using the named colon format (:param). This format is consistent regardless of database backend. To send values in for the parameters, we pass them into the Connection.execute() method as additional arguments.

In the example below we use the bound parameter *:first_name* in the query, and then pass in the value of the python variable *name* when executing the query.

"Rätt" version:

In [8]:
name = "Frida"

parameterized_query = text("Select * from Users where FirstName like '%' + :first_name + '%'")

with engine.connect() as conn:    
    result = conn.execute(parameterized_query, {"first_name": name})
    print(result.fetchall())

[('571110-3843', 'frieri', '7a981e17886344fb031e3735a7284b8c', 'Frida', 'Ericson', 'frida.ericson@hotmail.com', '0702-8579941')]


"Fel" version:

In [10]:
name = "';--"
 
unparameterized_query = text("Select * from Users where FirstName like '%" + name + "'%'")
 
with engine.connect() as conn:    
    result = conn.execute(unparameterized_query)
    print(result.fetchall())

[('500603-4268', 'johlen', '422cf6c6f212dde0fa96c532de240104', 'Johanna', 'Lennartsson', 'johanna.lennartsson@gmail.com', '070-9428041'), ('500607-6521', 'catknu', 'c5aa65949d20f6b20e1a922c13d974e7', 'Catharina', 'Knutson', 'catharina.knutson@yahoo.com', '0702-3351252'), ('530407-7989', 'ullalv', '4fec58181bb416f09f8ef0f69433584f', 'Ulla', 'Alvarsson', 'ulla.alvarsson@hotmail.com', '070-9922357'), ('530720-7675', 'alvlin', '2194506fc6ef7a2048f03a0f4ee7c641', 'Alvin', 'Lindholm', 'alvin.lindholm@gmail.com', '0701-6100069'), ('540430-4887', 'milalb', '3047ee053d45323e65192012176a2a1c', 'Milla', 'Albertsson', 'milla.albertsson@gmail.com', '0702-4265981'), ('550930-7164', 'mymard', '3587f76616df673c64f36e1d8babc2e7', 'My', 'Mårdh', 'my.mardh@yahoo.com', '0701-7763403'), ('561108-3389', 'vilmat', 'aa426df08f79c27a95d70a496a69759c', 'Vilhelmina', 'Matsson', 'vilhelmina.matsson@hotmail.com', '0701-8262655'), ('570501-4924', 'monsol', 'c1de2111b16e6b21b794451fe54ef86f', 'Mona', 'Solberg', 'mon

### Exercise
Fix the airport search application above, and make it invulnerable to SQL injection attacks.

For testing, you may try the search string: ';--

This string will select **ALL** rows in the unfixed version off the app.

When secured it should select **NO** rows, as no airport location contains the string: ';--

In [9]:
search_string = input("Search airports: ")

print(f"Search string: {search_string}\n")

safe_query = text("select top 10 IATA, [Airport name] as 'name', [Location served] as 'location' from airports where [Location served] like '%' + :search_string + '%'")


with engine.connect() as conn:    
    result = conn.execute(safe_query, {"search_string": search_string})

    print(f"{'IATA'.ljust(8)}{'Airport name'.ljust(50)}{'Location'}")
    ' a'
    for airport in result:
        print(f"{str(airport.IATA).ljust(8)}{str(airport.name).ljust(50)}{airport.location}")

    print(safe_query, '\n')


Search string: stock

IATA    Airport name                                      Location
ARN     Stockholm Arlanda Airport                         Stockholm, Sweden
BMA     Stockholm Bromma Airport                          Stockholm, Sweden
FST     Fort Stockton–Pecos County Airport                Fort Stockton, Texas, United States
NYO     Stockholm Skavsta Airport                         Stockholm / Nyköping, Sweden
RLG     Rostock–Laage Airport                             Rostock, Mecklenburg-Vorpommern, Germany
SCK     Stockton Metropolitan Airport                     Stockton, California, United States
SMP     Stockholm Airport[1]                              Stockholm, Papua New Guinea
STO     metropolitan area6                                Stockholm, Sweden
VST     Stockholm Västerås Airport                        Stockholm / Västerås, Sweden
select top 10 IATA, [Airport name] as 'name', [Location served] as 'location' from airports where [Location served] like '%' + :search_s